# Explore here

In [1]:
import numpy as np, random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import os.path
import joblib
import sklearn
import sklearn.metrics as metrics
import psycopg2
import sqlite3
import requests

In [2]:
df_rd= pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
df_rd.head(4)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106


In [3]:
df_rd2= pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')
df_rd2.head(4)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."


In [4]:
#df_rdMovies = df_rd.drop(['']) = pd.to_csv('../data/processed/PelisyCreditos.csv') #Dataframe con peliculas y creditos.
#df_rdPyCfilt = df_rd2.drop(['']) = pd.to_csv('../data/processed/ExtraInfo.csv')# Dataframe con Json filtrado a peticion.
conn = sqlite3.connect("../data/raw/tmdb5k.db")

df_rd.to_sql("pelis", conn, if_exists = "replace", index = False)
df_rd2.to_sql("creditos", conn, if_exists = "replace", index = False)

4803

In [5]:

query = """
    SELECT *
    FROM pelis
    INNER JOIN creditos
    ON pelis.title = creditos.title;
"""

df_rd3 = pd.read_sql_query(query, conn)
conn.close()

df_rd3 = df_rd3.drop_duplicates(keep='last')
df_rd3.to_csv('../data/processed/merged.csv')
df_rd3.head()
#budget	genres	homepage	id	keywords	original_language	original_title	overview	popularity	production_companies	...	runtime	spoken_languages	status	tagline	title	vote_average	vote_count	movie_id	cast	crew

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
df_rd3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [7]:
#filtramos por cada columna con formato diccionario para crear nuevas columnas con el class name como nombre de columna y su valor como contenido de la muestra.
# Crear un nuevo DataFrame vacío para almacenar los nuevos datos
df_rd3_dicts = df_rd3.copy()[['genres', 'keywords', 'cast']]
df_rd3_raw = df_rd3.copy()[['movie_id', 'title', 'overview', 'crew']]


#### Paso 3: Transforma los datos

Como puedes ver, hay algunas columnas con formato JSON. De cada uno de los JSONs, selecciona el atributo `name` y reemplaza las columnas `genres` y `keywords`. Para la columna `cast`, selecciona los tres primeros nombres.

Las únicas columnas que quedan por modificar son `crew` (equipo) y `overview` (resumen). Para la primera columna, transfórmala para que contenga el nombre del director. Para la segunda, conviértela en una lista.

Una vez hayamos terminado de procesar las columnas y que el modelo de recomendación no se confunda, por ejemplo, entre *Jennifer Aniston* y *Jennifer Conelly*, quitaremos los espacios entre las palabras. Aplica esta función a las columnas `genres`, `cast`, `crew` y `keywords`.

Por último, reduciremos nuestro conjunto de datos combinando todas nuestras columnas convertidas anteriores en una sola columna llamada `tags` (que crearemos). Esta columna ahora tendrá todos los elementos separados por comas y luego las reemplazaremos por espacios en blanco. Debería quedar algo así:


In [8]:
import pandas as pd
import json

def sacarvalor(json_str, limit=None):
    if pd.notna(json_str):
        data = json.loads(json_str)
        return [item['name'] for item in data[:limit]] if limit else [item['name'] for item in data]
    else:
        return None
df_rd3_dicts['cast'] = df_rd3_dicts['cast'].apply(sacarvalor, limit=3)
df_rd3_dicts['genres'] = df_rd3_dicts['genres'].apply(sacarvalor)
df_rd3_dicts['keywords'] = df_rd3_dicts['keywords'].apply(sacarvalor)

In [9]:
df_rd3_dicts.head()

,genres,keywords,cast
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]"
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]"


In [10]:

def sacardirector(json_str):
    try:
        data = json.loads(json_str)
        for member in data:
            if member['job'] == 'Director':
                return member['name']
        return None  # 
    except (json.JSONDecodeError, TypeError):
        print(f"Error al analizar el JSON: {json_str}")
        return None

df_rd3_raw['crew'] = df_rd3_raw['crew'].apply(sacardirector)


In [11]:
'''def delcolpos(df, posicion):
    df.drop(df.columns[posicion], axis=1, inplace=True)
    return df
contador = 0
while contador < 1:  # Cambia 1 por el número de veces que deseas eliminar
    df_rd3_raw = delcolpos(df_rd3_raw, 2)
    contador += 1'''
#Por algun motivo se me ha clonado la columna title y no soy capaz de quitarla de ninguna de las formas , intento quitar una 
#pero se me quitan las dos. Por nombre por posicion, cambiando el query de sql, i dont know.
df_rd3_raw.drop(['movie_id'], axis=1, inplace=True)

Como puedes ver, hay algunas columnas con formato JSON. De cada uno de los JSONs, selecciona el atributo `name` y reemplaza las columnas `genres` y `keywords`. Para la columna `cast`,
selecciona los tres primeros nombres.

Las únicas columnas que quedan por modificar son `crew` (equipo) y `overview` (resumen). 
Para la primera columna, transfórmala para que contenga el nombre del director. Para la segunda, conviértela en una lista.

Una vez hayamos terminado de procesar las columnas y que el modelo de recomendación no se confunda, por ejemplo, 
entre *Jennifer Aniston* y *Jennifer Conelly*, quitaremos los espacios entre las palabras. Aplica esta función a las columnas `genres`, `cast`, `crew` y `keywords`.

Por último, reduciremos nuestro conjunto de datos combinando todas nuestras columnas convertidas anteriores en una sola columna llamada `tags` (que crearemos). 
Esta columna ahora tendrá todos los elementos separados por comas y luego las reemplazaremos por espacios en blanco. Debería quedar algo así:


new_df["tags"][0]


In [12]:
import re
df_final = pd.DataFrame(columns=['tag', 'title'])
df_final['title'] = df_rd['title']
df_final['tag'] = (
    df_rd3_raw['crew'].astype(str) + " " +
    df_rd3_dicts['cast'].astype(str) + " " +
    df_rd3_dicts['keywords'].astype(str) + " " +
    df_rd3_dicts['genres'].astype(str) + " " +
    df_rd3_raw["overview"].astype(str)
)


def agregar_espacios(texto):
    # Eliminar comas y signos de puntuación, y agregar espacios entre palabras
    texto_limpio = re.sub(r"(\w)(\W)(\w)", r"\1 \3", texto)  # Inserta un espacio entre palabras
    texto_limpio = re.sub(r"[^\w\s]", "", texto_limpio)  # Elimina caracteres no alfanuméricos
    return texto_limpio

df_final['tag'] = df_final['tag'].apply(agregar_espacios)


In [13]:
del df_rd
del df_rd2
del df_rd3
del df_rd3_dicts
del df_rd3_raw


In [ ]:
#probamos el knearestnewighbours con countervectorizer.
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
vect = CountVectorizer()
Xvarspred = vect.fit_transform(df_final['tag'])

# Suponiendo que "title" es una categoría y la codificamos
y_pred = pd.factorize(df_final['title'])[0]

# Crear el modelo KNN
knn = KNeighborsClassifier(n_neighbors=6)

# Entrenar el modelo
knn.fit(Xvarspred, y_pred)

# Predecir el título de una nueva película
new_description = "Zombies"
new_vec = vect.transform([new_description])
predicted_title = knn.predict(new_vec)
predicted_title_index = knn.predict(new_vec)[0]
print("Título predicho:", df_final.iloc[predicted_title_index]['title'])


Título predicho: Pandaemonium


In [ ]:
#probamos el knearestnewighbours con la Nueva Feature BERT
from transformers import AutoTokenizer, AutoModel
from sklearn.neighbors import KNeighborsClassifier
import faiss
import torch
import nltk
from nltk.tokenize import word_tokenize
df_final['tag_tokenized'] = df_final['tag'].apply(word_tokenize)
# Cargar modelo pre-entrenado BERT
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')


def get_bert_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.pooler_output.numpy()


embeddings = get_bert_embeddings(df_final['tag_tokenized'])


index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


new_text = "Gente borracha"
new_embedding = get_bert_embeddings([new_text])


k = 5
distances, indices = index.search(new_embedding, k)


labels = [df_final['title'][i] for i in indices[0]]


predicted_label = max(set(labels), key=labels.count)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/vscode/nltk_data'
    - '/usr/local/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


Increible aventura, creo que he encontrado algo que no me gusta del DataScience. Me falta fluidez aun para las iteraciones de strings , cuesta visualizarlo